In [1]:
import pandas as pd

import joblib

### Evaluation of new models

In [2]:
logistic = pd.read_csv('../predictions/LogisticRegression_new.csv')
dt = pd.read_csv('../predictions/DecisionTreeClassifier_new.csv')
rf = pd.read_csv('../predictions/RandomForestClassifier_new.csv')
xgb = pd.read_csv('../predictions/XGBClassifier_new.csv')

In [3]:
races = logistic.Race.unique()
genders = logistic.Gender.unique()
races, genders

(array(['Black', 'White'], dtype=object),
 array(['Male', 'Female'], dtype=object))

### Demographic parity

In [4]:
def demo_parity(df: pd.DataFrame, col: str, val: str):
    total_num = len(df[df[col] == val])
    predict_true = len(df[(df['Loan_Status'] == 1) & (df[col] == val)])
    return predict_true/total_num

In [5]:
class Model:
    def __init__(self, name: str):
        self.name = name
        self.gender_ratio = 0
        self.race_ratio = 0
        self.fair_score = 0

    def __str__(self):
        return f"Model('{self.name}', fair_score={self.fair_score})"


In [6]:
races = logistic.Race.unique()
genders = logistic.Gender.unique()
races, genders

(array(['Black', 'White'], dtype=object),
 array(['Male', 'Female'], dtype=object))

In [7]:
def show_bias(df: pd.DataFrame, model: Model):
    demo_dict = {}
    print('Gender: ')
    for g in genders:
        demo_dict[g] = demo_parity(df, 'Gender', g)
        print(g, demo_dict[g])

    difference = abs(demo_dict["Male"] - demo_dict["Female"])
    ratio = demo_dict["Male"]/demo_dict["Female"] if demo_dict["Female"] != 0 else 0
    ratio = ratio if ratio < 1 else 1/ratio
    model.gender_ratio = ratio
    print('Difference: ', difference)
    print('Ratio: ', ratio)

    print()
    demo_dict = {}
    print('Race: ')
    for r in races:
        demo_dict[r] = demo_parity(df, 'Race', r)
        print(r, demo_dict[r])

    difference = abs(demo_dict["White"] - demo_dict["Black"])
    ratio = demo_dict["White"]/demo_dict["Black"] if demo_dict["Black"] != 0 else 0
    ratio = ratio if ratio < 1 else 1/ratio
    model.race_ratio = ratio
    print('Difference: ', difference)
    print('Ratio: ', ratio)

In [8]:
model_list_new = [Model('Logistic'), Model('DecisionTree'), Model('RandomForest'), Model('XGB')]

In [9]:
show_bias(logistic, model_list_new[0])

Gender: 
Male 0.7643097643097643
Female 0.7285714285714285
Difference:  0.035738335738335736
Ratio:  0.9532410320956576

Race: 
Black 0.8263473053892215
White 0.7
Difference:  0.12634730538922156
Ratio:  0.8471014492753624


In [10]:
show_bias(dt, model_list_new[1])

Gender: 
Male 0.7171717171717171
Female 0.6714285714285714
Difference:  0.04574314574314575
Ratio:  0.9362173038229376

Race: 
Black 0.7724550898203593
White 0.655
Difference:  0.11745508982035924
Ratio:  0.8479457364341085


In [11]:
show_bias(rf, model_list_new[2])

Gender: 
Male 0.7676767676767676
Female 0.6857142857142857
Difference:  0.0819624819624819
Ratio:  0.893233082706767

Race: 
Black 0.8203592814371258
White 0.695
Difference:  0.12535928143712582
Ratio:  0.8471897810218977


In [12]:
show_bias(xgb, model_list_new[3])

Gender: 
Male 0.7104377104377104
Female 0.6142857142857143
Difference:  0.0961519961519961
Ratio:  0.8646580907244414

Race: 
Black 0.7544910179640718
White 0.64
Difference:  0.1144910179640718
Ratio:  0.8482539682539684


#### Gender

In [13]:
model_list_new = sorted(model_list_new, key=lambda x: x.gender_ratio)
for model in model_list_new:
    print(f'{model.name}, {model.gender_ratio:.4f}')

XGB, 0.8647
RandomForest, 0.8932
DecisionTree, 0.9362
Logistic, 0.9532


#### Race

In [14]:
model_list_new = sorted(model_list_new, key=lambda x: x.race_ratio)
for model in model_list_new:
    print(f'{model.name}, {model.race_ratio:.4f}')

Logistic, 0.8471
RandomForest, 0.8472
DecisionTree, 0.8479
XGB, 0.8483


Here we use the average of `Race` and `Gender` demographic parity ratio as the bias score. The higher the score, the less biased the model is. 

In [15]:
for model in model_list_new:
    model.fair_score = (model.gender_ratio + model.race_ratio) / 2

In [16]:
print('\nNew Model List:\n')

model_list_new = sorted(model_list_new, key=lambda x: x.name)
for model in model_list_new:
    print(model)

print('avg fair score: ', sum([model.fair_score for model in model_list_new])/len(model_list_new))


New Model List:

Model('DecisionTree', fair_score=0.892081520128523)
Model('Logistic', fair_score=0.90017124068551)
Model('RandomForest', fair_score=0.8702114318643324)
Model('XGB', fair_score=0.856456029489205)
avg fair score:  0.8797300555418927


In [17]:
print('\nNew Model List:\n')

model_list_new = sorted(model_list_new, key=lambda x: x.name)
print('Race: ')
for model in model_list_new:
    print(f'{model.name}: {model.race_ratio:.4f}') 

avg_race_score  = sum([model.race_ratio for model in model_list_new])/len(model_list_new)
print('avg race score: ', avg_race_score)

print('\nGender: ')
for model in model_list_new:
    print(f'{model.name}: {model.gender_ratio:.4f}')

avg_gender_score  = sum([model.gender_ratio for model in model_list_new])/len(model_list_new)
print('avg gender score: ', avg_gender_score)


New Model List:

Race: 
DecisionTree: 0.8479
Logistic: 0.8471
RandomForest: 0.8472
XGB: 0.8483
avg race score:  0.8476227337463342

Gender: 
DecisionTree: 0.9362
Logistic: 0.9532
RandomForest: 0.8932
XGB: 0.8647
avg gender score:  0.9118373773374508
